In [1]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
from scipy.stats import norm
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy
import Tasmanian

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pickle
import gzip

warnings.simplefilter("ignore", category=RuntimeWarning)
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 2.38 s, sys: 356 ms, total: 2.73 s
Wall time: 2.82 s


[[-1.93137905e+18  7.48198939e+19 -3.14378961e+19  3.36598790e+19
  -5.31614543e+19]
 [ 8.29813641e+19  1.60964624e+18 -7.69155025e+19 -7.52940034e+19
  -6.76542954e+19]
 [ 4.21744129e+18 -7.42377057e+19  4.93604470e+19 -6.39590743e+19
   3.54068553e+18]
 [ 9.61946935e+19 -4.82617831e+19  6.34968589e+19  3.46123496e+19
  -1.92209996e+19]
 [-2.76297005e+19 -3.49415996e+18  8.99136858e+19 -1.58693659e+18
  -6.38886037e+19]
 [ 9.48648780e+19 -8.53343419e+19 -3.62487286e+19  8.81286170e+18
  -6.68813829e+19]
 [-7.10965008e+19  1.54445010e+19  2.05978848e+19  1.04886677e+19
   4.63528712e+19]
 [ 4.71202143e+19 -1.60952226e+19  4.13533817e+19 -8.12722255e+19
  -1.74925931e+19]
 [ 1.98418260e+19  8.20702860e+19 -3.32160294e+19  9.67158263e+19
  -1.52590733e+18]
 [-2.91959024e+19 -4.12546260e+19 -2.61082421e+19  2.15814088e+19
  -3.46626761e+19]
 [-8.21375997e+19 -6.32864357e+19  6.14542128e+19 -4.85072918e+19
  -9.47448994e+19]
 [ 4.60650704e+19  1.43562960e+19  5.05310404e+19  2.93799683e+19

In [7]:
#Make fake data
import numpy as np

def ei(random_var):
    ei = np.exp(random_var)
    return ei

In [21]:
#Test MC integration
def mc_integrate(func, a, b, dim, n = 1000):
    # Monte Carlo integration of given function over domain from a to b (for each parameter)
    # dim: dimensions of function
    
    random_var = np.random.uniform(a, b, (n, dim))
    y = func(random_var)
    
    y_mean = np.average(y) #y.sum()/len(y)
    domain = np.power(b-a, dim)
    
    integ = domain * y_mean
    
    return integ

print(mc_integrate(ei,-1, 1, 11, n = 1000))

2425.5292076225082


In [ ]:
#Test Sparse Grid Integration